# Connecting to mysql server and query the data

## Libraries and settings

In [1]:
# Libraries
import os
import mysql.connector
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

# Show current working directory
print(os.getcwd())

/workspaces/data_analytics_project


## Connect to database

In [2]:
# Create data base
#conn = mysql.connector.connect()
conn = mysql.connector.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    password = '1234')
    #database = 'movie_database') 
#cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.csv')
for i in flist:
    print(i)

InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (111 Connection refused)

## Read data from file

In [31]:
df = pd.read_csv('IMDB Top 250 Movies.tsv', sep=';', encoding='utf-8')
df.head(5)

,title,genres,release_year,runtime_(mins),age_rating,director,movie_rating,votes_count
0,The Shawshank Redemption,Drama,1994,142,12,Frank Darabont,"9,3",2969404
1,The Godfather,"Crime, Drama",1972,175,16,Francis Ford Coppola,"9,2",2070969
2,The Godfather Part II,"Crime, Drama",1974,202,16,Francis Ford Coppola,"9,0",1398083
3,The Dark Knight,"Action, Crime, Drama, Thriller",2008,152,16,Christopher Nolan,"9,0",2950798
4,12 Angry Men,"Crime, Drama",1957,96,12,Sidney Lumet,"9,0",895706


## Convert data to appropriate format

In [32]:
# Rename selected columns
df = df.rename(columns={'runtime_(mins)': 'runtime'})
df.head(5)

,title,genres,release_year,runtime,age_rating,director,movie_rating,votes_count
0,The Shawshank Redemption,Drama,1994,142,12,Frank Darabont,"9,3",2969404
1,The Godfather,"Crime, Drama",1972,175,16,Francis Ford Coppola,"9,2",2070969
2,The Godfather Part II,"Crime, Drama",1974,202,16,Francis Ford Coppola,"9,0",1398083
3,The Dark Knight,"Action, Crime, Drama, Thriller",2008,152,16,Christopher Nolan,"9,0",2950798
4,12 Angry Men,"Crime, Drama",1957,96,12,Sidney Lumet,"9,0",895706


## Create SQL-table in the database

In [33]:
cursor.execute('''CREATE TABLE IF NOT EXISTS movies_table (
               title VARCHAR(200),
               genres VARCHAR(200),
               release_year INT(3),
               runtime INT(3),
               age_rating INT(2),
               director VARCHAR(200),
               movie_rating FLOAT(2),
               votes_count INT(10)
               )''')     
# Confirm changes to the table
conn.commit()

## Write data to table in db

In [34]:
df.to_sql(name = 'movies_table',
          con = conn,
          index = False,
          if_exists = 'replace')

250

## Query the table

In [35]:
# Query the SQL-table
cursor.execute('''SELECT * from movies_table where genres like '%Crime%' limit 5''')

columns = df.columns

df = pd.DataFrame(cursor.fetchall(), 
                  columns=columns)
df

,title,genres,release_year,runtime,age_rating,director,movie_rating,votes_count
0,The Godfather,"Crime, Drama",1972,175,16,Francis Ford Coppola,"9,2",2070969
1,The Godfather Part II,"Crime, Drama",1974,202,16,Francis Ford Coppola,"9,0",1398083
2,The Dark Knight,"Action, Crime, Drama, Thriller",2008,152,16,Christopher Nolan,"9,0",2950798
3,12 Angry Men,"Crime, Drama",1957,96,12,Sidney Lumet,"9,0",895706
4,Pulp Fiction,"Crime, Drama",1994,154,16,Quentin Tarantino,"8,9",2279743


## Close db connection (if open)

In [36]:
# Close db connection (if open)
try:
    if check_conn(conn):
        conn.close()
    else:
        pass
except:
    pass

# Status (True = open, False = closed)
print(check_conn(conn))

False


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [37]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.5.0-1025-azure
Datetime: 2024-12-05 16:13:30
Python Version: 3.11.10
-----------------------------------
